In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image, image_dataset_from_directory
from keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import imagenet_utils
import numpy as np
import os
import json
import csv
import matplotlib.pyplot as plt
import cv2
from google.colab.patches import cv2_imshow # cv2.imshow does not work on Google Colab notebooks, --> use cv2_imshow instead

In [ ]:
# loading settings
model_string = "resnet50"  # inceptionv3, resnet50, vgg16
dataset_string = "imagenet"  # imagenet


# other settings
heatmap_intensity = 0.5
base_learning_rate = 0.0001


# data set

#default one, labels via folder structure, 1006 images (600 for training), 214 MB total
#path_to_dataset = '/content/gdrive/My Drive/What are CNNs looking at/New Masks Dataset/' 

#augmented ??
#path_to_dataset = '/content/gdrive/My Drive/What are CNNs looking at/files/Face Mask Dataset/' 

#multiple masks per image, bounding boxes, xml-labels, 853 images, 399 MB
#path_to_dataset = '/content/gdrive/My Drive/What are CNNs looking at/files/Face_Mask_Detection/images/' 

# former Face Mask Detection Dataset
# multiple masks per image, bounding boxes, csv/json-labels, 4326 images, <3 GB
# path_to_dataset = '/content/gdrive/My Drive/What are CNNs looking at/files/Multi Face Masks per Image clean/images' 

# one mask per image, labels via folder structure, 12k files (10k for training), 329 MB total
path_to_dataset = '/content/gdrive/My Drive/What are CNNs looking at/files/12k Face Mask Dataset/' 

# todo: description
#path_to_dataset = '/content/gdrive/My Drive/Kaggle/files/New_Masks_Dataset'

# deprecated
# multiple masks per image, bounding boxes, csv/json-labels, 6024 images (4326 usable because of missing labels), 3 GB
#path_to_dataset = '/content/gdrive/MyDrive/What are CNNs looking at/files/Face Mask Detection Dataset/Medical mask/Medical mask/Medical Mask/images' 



other_path = "/content/gdrive/My Drive/What are CNNs looking at/New Masks Dataset/"



In [ ]:
# model dependent imports, weight loading and model dependent parameter setting
K.clear_session()
if model_string == "inceptionv3":
    from tensorflow.keras.applications.inception_v3 import InceptionV3
    from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions
    Model = InceptionV3
    if dataset_string == "imagenet":
        model = InceptionV3(weights='imagenet')
    target_input_dimension = 299
    heatmap_dimension = 8
    last_layer_name = 'conv2d_93'
elif model_string == "resnet50":
    from tensorflow.keras.applications import ResNet50
    from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
    Model = ResNet50
    if dataset_string == "imagenet":
        #model = ResNet50(weights="imagenet")
        model = ResNet50()
    target_input_dimension = 224
    heatmap_dimension = 7
    last_layer_name = 'conv5_block3_3_conv'
elif model_string == "vgg16":
    from tensorflow.keras.applications import VGG16
    from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
    Model = VGG16
    if dataset_string == "imagenet":
        model = VGG16(weights="imagenet")
    target_input_dimension = 224
    heatmap_dimension = 14
    last_layer_name = 'block5_conv3'
else:  # use InceptionV3 and imagenet as default
    from tensorflow.keras.applications.inception_v3 import InceptionV3
    from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions
    if dataset_string == "imagenet":
        model = InceptionV3(weights='imagenet')
    target_input_dimension = 299
    heatmap_dimension = 8
    last_layer_name = 'conv2d_93'

In [ ]:
def gradCAM(orig, model, model_string, DIM, HM_DIM, last_layer, classes, intensity=0.5, res=250):
    img = image.load_img(orig, target_size=(DIM, DIM))

    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    preds = model.predict(x)
    for pred_it, label_it in zip(preds[0], classes):
      print(label_it, ": ", pred_it)
    print("preds")
    print(preds)
    prob = np.max(preds[0])
    index = list(preds[0]).index(prob)
    label = classes[index]
    label = "{}: {:.2f}%".format(label, prob * 100)
    print("[INFO] {}".format(label))

    with tf.GradientTape() as tape:
        last_conv_layer = model.get_layer(last_layer)
        iterate = tf.keras.models.Model([model.inputs], [model.output, last_conv_layer.output]) # run model and achive certain output 'last_conv_layer.output'
        model_out, last_conv_layer = iterate(x)
        class_out = model_out[:, np.argmax(model_out[0])]
        grads = tape.gradient(class_out, last_conv_layer) # class out: take derivative; last_conv_layer: variable to derive from
        pooled_grads = K.mean(grads, axis=(0, 1, 2))

    heatmap = tf.reduce_mean(tf.multiply(pooled_grads, last_conv_layer), axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)
    heatmap = heatmap.reshape((HM_DIM, HM_DIM))

    img_original = cv2.imread(orig)
    heatmap = cv2.resize(heatmap, (img_original.shape[1], img_original.shape[0]))
    # plt.matshow(heatmap)
    # plt.show()

    heatmap = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
    # plt.matshow(heatmap)
    # plt.show()
    img_heatmap = heatmap * intensity + img_original

    #cv2_imshow(img_heatmap)
    return img_heatmap

In [ ]:
# mount google drive to access database

from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cd $path_to_dataset

In [ ]:
# import database - labels from file system

train_dir = os.path.join(path_to_dataset, 'Train')
validation_dir = os.path.join(path_to_dataset, 'Validation')
test_dir = os.path.join(path_to_dataset, 'Test')
BATCH_SIZE = 32
IMG_SIZE = (224, 224)

train_dataset = image_dataset_from_directory(train_dir,
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE)
validation_dataset = image_dataset_from_directory(validation_dir,
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE,
                                                  image_size=IMG_SIZE)
test_dataset = image_dataset_from_directory(test_dir,
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE,
                                                  image_size=IMG_SIZE)


In [ ]:
# import database - labels from file system - no validation data

train_dir = os.path.join(path_to_dataset, 'Train')
test_dir = os.path.join(path_to_dataset, 'Test')
BATCH_SIZE = 32
IMG_SIZE = (224, 224)

train_dataset = image_dataset_from_directory(train_dir,
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE)
test_dataset = image_dataset_from_directory(test_dir,
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE,
                                                  image_size=IMG_SIZE)
validation_dataset = test_dataset

In [ ]:
class_names = train_dataset.class_names
print(class_names)

In [ ]:
# data augmntation 

data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'), # horizontal_and_vertical, horizontal
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

In [ ]:
# observe train_dataset and image_dataset_from_directory
print(type(train_dataset)) #tensorflow.python.data.ops.dataset_ops.BatchDataset
print(train_dataset.class_names)
#print(np.shape(train_dataset.images)) #(batch_size, image_size[0], image_size[1], num_channels)

In [ ]:
# define list of example images (e.g. for heatmap)

if path_to_dataset == '/content/gdrive/My Drive/What are CNNs looking at/New Masks Dataset/':
  image_dir_mask = 'Test/Mask'
  image_dir_no_maks = 'Test/Non Mask'
  image_names_mask = ['2070.jpg', '2190.png', '2222.png', '2268.png']
  image_names_no_maks = ['real_01033.jpg', 'real_01057.jpg', 'real_01061.jpg', 'real_01081.jpg']

elif path_to_dataset == '/content/gdrive/My Drive/What are CNNs looking at/files/12k Face Mask Dataset/':
  image_dir_mask = 'Test/WithMask'
  image_dir_no_maks = 'Test/WithoutMask'
  image_names_mask = ['1175.png', '1362.png', '1404.png', '1439.png', '190.png']
  image_names_no_maks = ['1.png', '1407.png', '2246.png', '2871.png', '3574.png']

else:
  image_dir_mask = ''
  image_dir_no_maks = ''
  image_names_mask = []
  image_names_no_maks = []
  print("Warning! No sample images prepared!")

In [ ]:
# define list of example images from other database (e.g. for heatmap)

if other_path == '/content/gdrive/My Drive/What are CNNs looking at/New Masks Dataset/':
  image_dir_mask2 = 'Test/Mask'
  image_dir_no_maks2 = 'Test/Non Mask'
  image_names_mask2 = ['2070.jpg', '2190.png', '2222.png', '2268.png']
  image_names_no_maks2 = ['real_01033.jpg', 'real_01057.jpg', 'real_01061.jpg', 'real_01081.jpg']

elif other_path == '/content/gdrive/My Drive/What are CNNs looking at/files/12k Face Mask Dataset/':
  image_dir_mask2 = 'Test/WithMask'
  image_dir_no_maks2 = 'Test/WithoutMask'
  image_names_mask2 = ['1175.png', '1362.png', '1404.png', '1439.png', '190.png']
  image_names_no_maks2 = ['1.png', '1407.png', '2246.png', '2871.png', '3574.png']

else:
  image_dir_mask2 = ''
  image_dir_no_maks2 = ''
  image_names_mask2 = []
  image_names_no_maks2 = []
  print("Warning! No sample images prepared!")

In [ ]:
# creating the model

# currently ResNet50 is hard coded
K.clear_session()

# load ResNet50 without dense layers
base_model = ResNet50(input_shape=(target_input_dimension, 
                               target_input_dimension, 
                               3), 
                  include_top=False, 
                  weights='imagenet')
base_model.trainable = False

# create 2 dense layers
pooling_layer_2d = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(2, activation='softmax')  # 2 output classes
# with 2 output neurons there is a reshaping error
# using 1 output neuron works but this is less general

# create new model (add dense layers to convolutional model 'base_model')
inputs = tf.keras.Input(shape=(target_input_dimension, 
                               target_input_dimension, 
                               3))
x = preprocess_input(inputs)
#x = data_augmentation(x)
x = base_model(x, training=False)  # Why training=False?
x = tf.identity(x) # needed to be able to obtain heatmap
x = pooling_layer_2d(x)
outputs = prediction_layer(x)

model = tf.keras.Model(inputs, outputs)

#model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
#              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
#              metrics=['accuracy'])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])
# using from_logits=False because we use a softmax activation in the last layer
# which provices a non-logit output in the range [0, 1]

In [ ]:
model.summary()

In [ ]:
# bench marking
loss0, accuracy0 = model.evaluate(validation_dataset)
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

In [ ]:
# heatmap bench mark

downsize_factor = 1 # >=1

for imagename in image_names_mask:
  heatmap = gradCAM(image_dir_mask+'/'+imagename, model, model_string, target_input_dimension, heatmap_dimension, 'tf.identity',class_names)
  heatmap = cv2.resize(heatmap, (int(heatmap.shape[1]/downsize_factor), int(heatmap.shape[0]/downsize_factor)))
  cv2_imshow(heatmap)

for imagename in image_names_no_maks:
  heatmap = gradCAM(image_dir_no_maks+'/'+imagename, model, model_string, target_input_dimension, heatmap_dimension, 'tf.identity', class_names)
  heatmap = cv2.resize(heatmap, (int(heatmap.shape[1]/downsize_factor), int(heatmap.shape[0]/downsize_factor)))
  cv2_imshow(heatmap)

In [ ]:
# retrain

initial_epochs = 5
history = model.fit(train_dataset,
                    epochs=initial_epochs,
                    validation_data=validation_dataset)

In [ ]:
# heatmap validation after retraining

downsize_factor = 1 # >=1

for imagename in image_names_mask:
  heatmap = gradCAM(image_dir_mask+'/'+imagename, model, model_string, target_input_dimension, heatmap_dimension, 'tf.identity',class_names)
  heatmap = cv2.resize(heatmap, (int(heatmap.shape[1]/downsize_factor), int(heatmap.shape[0]/downsize_factor)))
  cv2_imshow(heatmap)

for imagename in image_names_no_maks:
  heatmap = gradCAM(image_dir_no_maks+'/'+imagename, model, model_string, target_input_dimension, heatmap_dimension, 'tf.identity', class_names)
  heatmap = cv2.resize(heatmap, (int(heatmap.shape[1]/downsize_factor), int(heatmap.shape[0]/downsize_factor)))
  cv2_imshow(heatmap)

In [ ]:
# plot history of training and evaluate with test dataset

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_dataset)

In [ ]:
# apply other images

downsize_factor = 2 # >1

for imagename in image_names_mask2:
  heatmap = gradCAM(other_path+image_dir_mask2+'/'+imagename, model, model_string, target_input_dimension, heatmap_dimension, 'tf.identity',class_names)
  heatmap = cv2.resize(heatmap, (int(heatmap.shape[1]/downsize_factor), int(heatmap.shape[0]/downsize_factor)))
  cv2_imshow(heatmap)

for imagename in image_names_no_maks2:
  heatmap = gradCAM(other_path+image_dir_no_maks2+'/'+imagename, model, model_string, target_input_dimension, heatmap_dimension, 'tf.identity', class_names)
  heatmap = cv2.resize(heatmap, (int(heatmap.shape[1]/downsize_factor), int(heatmap.shape[0]/downsize_factor)))
  cv2_imshow(heatmap)

In [ ]:
# retrain including convolutional layers

model.layers[3].trainable = True

initial_epochs = 20
history = model.fit(train_dataset,
                    epochs=initial_epochs,
                    validation_data=validation_dataset)

In [ ]:
# heatmap validation after retraining

downsize_factor = 1 # >=1

for imagename in image_names_mask:
  heatmap = gradCAM(image_dir_mask+'/'+imagename, model, model_string, target_input_dimension, heatmap_dimension, 'tf.identity',class_names)
  heatmap = cv2.resize(heatmap, (int(heatmap.shape[1]/downsize_factor), int(heatmap.shape[0]/downsize_factor)))
  cv2_imshow(heatmap)

for imagename in image_names_no_maks:
  heatmap = gradCAM(image_dir_no_maks+'/'+imagename, model, model_string, target_input_dimension, heatmap_dimension, 'tf.identity', class_names)
  heatmap = cv2.resize(heatmap, (int(heatmap.shape[1]/downsize_factor), int(heatmap.shape[0]/downsize_factor)))
  cv2_imshow(heatmap)

In [ ]:
# plot history of training and evaluate with test dataset

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_dataset)

In [ ]:
# apply other images

downsize_factor = 2 # >1

for imagename in image_names_mask2:
  heatmap = gradCAM(other_path+image_dir_mask2+'/'+imagename, model, model_string, target_input_dimension, heatmap_dimension, 'tf.identity',class_names)
  heatmap = cv2.resize(heatmap, (int(heatmap.shape[1]/downsize_factor), int(heatmap.shape[0]/downsize_factor)))
  cv2_imshow(heatmap)

for imagename in image_names_no_maks2:
  heatmap = gradCAM(other_path+image_dir_no_maks2+'/'+imagename, model, model_string, target_input_dimension, heatmap_dimension, 'tf.identity', class_names)
  heatmap = cv2.resize(heatmap, (int(heatmap.shape[1]/downsize_factor), int(heatmap.shape[0]/downsize_factor)))
  cv2_imshow(heatmap)

## Archive

### Observe Data Augmentation

In [ ]:
# test augmentation 1

data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.5),
])

plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
  plt.imshow(images[0].numpy().astype("uint8"))
  plt.title(class_names[labels[0]])
  plt.axis("off")
  plt.figure(figsize=(10, 10))
  for i in range(9):
    raw_image = np.expand_dims(images[0].numpy().astype("uint8"), axis=0)
    augmented_image = data_augmentation(raw_image)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_image[0])
    plt.axis("off")

### from zip to files

In [ ]:
import zipfile
all_files=['/content/gdrive/My Drive/What are CNNs looking at/files/12k Face Mask Dataset/archive.zip']
#all_files=['face-mask-detection.zip', 'face-mask-12k-images-dataset.zip', 'covid-face-mask-detection-dataset.zip']
for file in all_files:
  zip_ref = zipfile.ZipFile(file, 'r')
  zip_ref.extractall('/content/gdrive/My Drive/What are CNNs looking at/files/12k Face Mask Dataset/')
  zip_ref.close()

### separate labeled/unlabeled images from 'Face Mask Detection Dataset'

In [ ]:
# separate images
# to be run only once - already run -> finished

import shutil

path_to_dataset = "/content/gdrive/MyDrive/What are CNNs looking at/files/Face Mask Detection Dataset/Medical mask/Medical mask/Medical Mask/images"

dirlist = os.listdir(path_to_dataset)
image_list = []
print(path_to_dataset)
print(len(dirlist))
i=0
for f in dirlist:
  sourcename = path_to_dataset+"/"+f
  targetname = "/content/gdrive/MyDrive/What are CNNs looking at/files/Multi Face Masks per Image clean/images/"+f
  print(i)
  if os.path.isfile(os.path.join(path_to_dataset, "../annotations/", f+".json")):
    image_list.append(f)
    shutil.copyfile(sourcename, targetname)
  i = i+1
print(len(image_list))

### import from csv

In [ ]:
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

In [ ]:
# watch images from list_ds
for f in list_ds.take(5):
  img = cv2.imread(f.numpy().decode("utf-8"))
  img = cv2.resize(img, (int(img.shape[1]/2), int(img.shape[0]/2)))
  cv2_imshow(img)

In [ ]:
def translate_label(old_label):
  mask_labels = ["mask_colorful", "mask_surgical"]
  no_mask_labels = ["turban", "helmet", "sunglasses", "eyeglasses", "hair_net", "hat", "goggles", "hood"]
  other_covering_label = ["hijab_niqab", "scarf_bandana", "balaclava_ski_mask", "face_shield", "gas_mask"]
  if old_label in mask_labels:
    new_label = "face_with_mask"
  elif old_label in no_mask_labels:
    new_label = "face_no_mask"
  elif old_label in other_covering_label:
    new_label = "face_other_covering"
  return new_label

In [ ]:
# import database with labels from csv

if path_to_dataset != '/content/gdrive/My Drive/What are CNNs looking at/files/Multi Face Masks per Image clean/images':
  print('This cell is not designed for the selected data base')
  raise StopExecution

label_file_name = "/content/gdrive/MyDrive/What are CNNs looking at/files/Multi Face Masks per Image clean/train.csv"
valid_classes = ["face_no_mask", "face_with_mask_incorrect", "face_with_mask", "face_other_covering"]
filelist = os.listdir(path_to_dataset)
no_of_images = len(filelist)
unique_filenames = [-1] * no_of_images
labels_string = ["" for i in range(no_of_images)]

with open(label_file_name, newline='') as csvfile:
     reader = csv.DictReader(csvfile)
     for row in reader:
         if row['name'] not in unique_filenames and row['classname'] in valid_classes:
             pos = filelist.index(row['name'])
             unique_filenames[pos] = row['name']
             labels_string[pos] = row['classname']
        
with open(label_file_name, newline='') as csvfile:
     reader = csv.DictReader(csvfile)
     for row in reader:
         if row['name'] not in unique_filenames:
             label = translate_label(row['classname'])
             pos = filelist.index(row['name'])
             unique_filenames[pos] = row['name']
             labels_string[pos] = label

# labels from string to int
labels_int = []
for label in labels_string:
  labels_int.append(valid_classes.index(label))
  if label == "":
    print("Error! Empty label found: ")

full_dataset = image_dataset_from_directory('/content/gdrive/My Drive/What are CNNs looking at/files/Multi Face Masks per Image clean/images', labels=labels_int, label_mode='int')

print(full_dataset.class_names)

### Other

In [ ]:
# temp

data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])



for images, labels in train_dataset.take(1):
  i = 31
  plt.imshow(images[i].numpy().astype("uint8"))
  plt.title(class_names[labels[i]])
  plt.axis("off")

  plt.figure(figsize=(10, 10))
  for k in range(9):
    augmented_image = data_augmentation(images[i])
    ax = plt.subplot(3, 3, k + 1)
    plt.imshow(augmented_image[0].numpy().astype("uint8"))
    plt.axis("off")

In [ ]:
# temp
for images, labels in train_dataset.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
# access layers

# access resnet layers
#model.layers[3].layers
#model.layers[3].get_layer("conv5_block3_3_conv")
#model.get_layer("resnet50").get_layer("conv5_block3_3_conv")

#model.trainable
#model.layers[3].trainable